In [17]:
import sys, os, importlib, json, boto3
import rasterio, geojson, folium, h3

import pandas as pd
import geopandas as gpd

from shapely.geometry import Point, mapping

sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")
import GOSTRocks.rasterMisc as rMisc
import GOSTRocks.ntlMisc as ntl
from GOSTRocks.misc import tPrint

%load_ext autoreload
%autoreload 2

from survey_extractor import survey_extract


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
survey_folder = '/home/wb411133/projects/ZON_DECSI_EnterpriseSurveys/Data/SURVEYS'
# The IDN data does not have coordinates
#idn_sample = os.path.join(survey_folder, "Firm", "IDN admin data 2017 sample.dta")
wbes_sample = os.path.join(survey_folder, "Firm", "WBES geocodes_sample.dta")
mwi_sample = os.path.join(survey_folder, "Malawi", "householdgeovariablesihs4.dta")

#Geospatial data
fathom_folder = '/home/public/Data/GLOBAL/FLOOD_SSBN/v2_2019'
fathom_folder_old = '/home/public/Data/GLOBAL/FLOOD_SSBN/v1_2016'
urban_layer = "/home/public/Data/GLOBAL/URBAN/GHS/GHS_SMOD/GHS_SMOD_E2020_GLOBE_R2022A_54009_1000_V1_0.tif"

# Output
results_folder = '/home/wb411133/projects/ZON_DECSI_EnterpriseSurveys/Data/RESULTS'
wbes_ntl_results = os.path.join(results_folder, "wbes_ntl_results.csv")
mwi_ntl_results = os.path.join(results_folder, "mwi_ntl_results.csv")

wbes_flood_results = os.path.join(results_folder, "wbes_flood_results.csv")
mwi_flood_results = os.path.join(results_folder, "mwi_flood_results.csv")

In [3]:
def gpd_read_stata(inFile, lat_col, lon_col):
    inD = pd.read_stata(inFile)
    geoms = [Point(x) for x in zip(inD[lon_col], inD[lat_col])]
    inD = gpd.GeoDataFrame(inD, geometry=geoms, crs=4326)
    return(inD)
wbesD = gpd_read_stata(wbes_sample, "lat_mask", "lon_mask")
mwiD = gpd_read_stata(mwi_sample, "lat_modified", "lon_modified")
mwiD = mwiD[~mwiD['lat_modified'].isnull()]
mwiD

,case_id,hhid,lat_modified,lon_modified,dist_road,dist_popcenter,dist_admarc,dist_auction,dist_boma,dist_borderpost,...,anntot_avg,wetQ_avg,wetQ_avgstart,h2015_tot,h2015_wetQ,h2015_wetQstart,h2016_tot,h2016_wetQ,h2016_wetQstart,geometry
0,301025230225,0001c970eecf473099368557e2080b3e,-14.683761,34.915074,13.0,32.0,17.0,125.0,44.0,39.0,...,808.0,631.0,17.0,1007.0,889.0,17.0,614.0,478.0,17.0,POINT (34.91507 -14.68376)
1,210374850204,000509f5cfcc4b078a09672b09425e95,-14.005029,33.794591,2.0,3.0,3.0,3.0,3.0,74.0,...,812.0,630.0,16.0,783.0,643.0,17.0,638.0,487.0,18.0,POINT (33.79459 -14.00503)
2,311057710075,000bc107780044e59327dbf7ec960ac1,-16.826165,35.269503,1.0,12.0,2.0,115.0,12.0,36.0,...,724.0,593.0,16.0,896.0,821.0,17.0,585.0,366.0,17.0,POINT (35.26950 -16.82616)
3,312048040073,000d1d26325d4f73a2ffbb8a99ab4752,-15.004730,35.163219,1.0,7.0,10.0,89.0,23.0,70.0,...,790.0,615.0,16.0,1079.0,960.0,17.0,690.0,497.0,17.0,POINT (35.16322 -15.00473)
4,311097790117,00104e33315844fdb2b8c6fdd35912a1,-17.016698,35.079629,15.0,23.0,15.0,135.0,23.0,21.0,...,692.0,565.0,16.0,843.0,784.0,17.0,490.0,315.0,13.0,POINT (35.07963 -17.01670)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12439,205053630064,ffe8b3cb44e646c4ba7e444c810b5262,-14.038697,34.511056,0.0,29.0,22.0,80.0,29.0,44.0,...,787.0,626.0,17.0,867.0,776.0,16.0,607.0,483.0,18.0,POINT (34.51106 -14.03870)
12440,307106820173,ffe9da89ba74465ba9be8a7e8cbd77ce,-15.959665,35.034930,2.0,15.0,4.0,13.0,20.0,41.0,...,794.0,621.0,16.0,1063.0,912.0,16.0,578.0,424.0,17.0,POINT (35.03493 -15.95966)
12441,208074540130,ffe9e824c41f45c4aa5efb80d3ee9603,-14.263875,34.518640,2.0,22.0,5.0,85.0,22.0,23.0,...,806.0,640.0,17.0,899.0,807.0,16.0,608.0,467.0,17.0,POINT (34.51864 -14.26388)
12442,312048040036,fffaa977174c43d79793cf945b57f948,-15.004730,35.163219,1.0,8.0,12.0,91.0,23.0,69.0,...,789.0,615.0,16.0,1071.0,953.0,17.0,687.0,495.0,17.0,POINT (35.16322 -15.00473)


In [14]:
wbesD.head()

,idstd,lat_mask,lon_mask,survey,WBCode,type,geometry
0,531201.0,23.719870,90.418999,Bangladesh2013,BGD,Enterprise Survey,POINT (90.41900 23.71987)
1,531202.0,23.752016,90.347771,Bangladesh2013,BGD,Enterprise Survey,POINT (90.34777 23.75202)
2,531203.0,23.711718,90.345955,Bangladesh2013,BGD,Enterprise Survey,POINT (90.34595 23.71172)
3,531204.0,23.802555,90.423981,Bangladesh2013,BGD,Enterprise Survey,POINT (90.42398 23.80256)
4,531205.0,23.802618,90.390488,Bangladesh2013,BGD,Enterprise Survey,POINT (90.39049 23.80262)


In [8]:
# get list of nighttime light files for use in survey_extractor
ntl_files = ntl.aws_search_ntl()
ntl_files = [[x, 'ntl_' + os.path.basename(x).split("_")[2][:8]] for x in ntl_files]

In [26]:
# Generate list of fathom_flood files for BGD and MWI
return_period = 100
mwi_fluvial_flood_file = os.path.join(fathom_folder, "malawi", "fluvial_undefended", f"FU_1in{return_period}.tif")
mwi_pluvial_flood_file = os.path.join(fathom_folder, "malawi", "pluvial", f"P_1in{return_period}.tif")
mwi_flood_files = [
                    [mwi_fluvial_flood_file, f'fluvial_{return_period}'],
                    [mwi_pluvial_flood_file, f'pluvial_{return_period}']
                  ]


fluvial_flood_file = os.path.join(fathom_folder_old, "bangladesh", "BD_fluvial_undefended", f"BD-FU-{return_period}-1.tif")
pluvial_flood_file = os.path.join(fathom_folder_old, "bangladesh", "BD_pluvial_undefended", f"BD-PU-{return_period}-1.tif")
bgd_flood_files = [
                    [fluvial_flood_file, f'fluvial_{return_period}'],
                    [pluvial_flood_file, f'pluvial_{return_period}']
                  ]    

In [ ]:
# Define other files to summarize
other_files = [
    [urban_layer, "ghsmod_2022"],        
]

# Run zonal stats

In [27]:
extractor = survey_extract(wbesD, "lat_mask", "lon_mask", "idstd")
flood_res = extractor.calc_zonal(bgd_flood_files)
flood_res

,idstd,fluvial_100,pluvial_100
0,531201.0,0.0,0.0000
1,531202.0,999.0,999.0000
2,531203.0,0.0,0.0000
3,531204.0,0.0,0.0000
4,531205.0,0.0,0.1031
...,...,...,...
1437,532638.0,0.0,0.0000
1438,532639.0,0.0,0.0000
1439,532640.0,0.0,0.0000
1440,532641.0,0.0,0.3550


In [28]:
ntl_res = extractor.calc_zonal(ntl_files)
ntl_res

,idstd,ntl_20120119,ntl_20120201,ntl_20120301,ntl_20120401,ntl_20120501,ntl_20120601,ntl_20120701,ntl_20120801,ntl_20120901,...,ntl_20211101,ntl_20211201,ntl_20220101,ntl_20220201,ntl_20220301,ntl_20220401,ntl_20220501,ntl_20220701,ntl_20221101,ntl_20221201
0,531201.0,25.350000,20.670000,17.690001,19.610001,18.540001,15.13,0.00,4.14,8.73,...,30.860001,32.090000,38.049999,32.860001,35.320000,27.760000,26.690001,30.15,36.619999,31.719999
1,531202.0,5.010000,5.200000,4.490000,3.740000,3.280000,2.59,0.00,4.79,3.78,...,10.680000,10.530000,12.990000,12.180000,12.450000,12.150000,13.540000,0.00,12.470000,11.760000
2,531203.0,2.880000,3.450000,2.320000,1.650000,1.770000,1.32,0.00,5.13,2.91,...,3.200000,4.730000,5.270000,5.410000,8.320000,7.020000,5.940000,0.00,10.750000,4.400000
3,531204.0,22.559999,22.040001,13.160000,13.630000,10.560000,13.99,0.00,7.98,11.64,...,21.770000,22.320000,27.299999,21.480000,24.700001,20.129999,26.459999,0.00,26.910000,23.920000
4,531205.0,14.690000,11.720000,10.390000,9.390000,7.920000,5.07,0.00,4.65,10.34,...,22.870001,21.600000,26.549999,22.540001,23.740000,20.610001,21.370001,31.68,23.900000,20.850000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1437,532638.0,24.480000,19.440001,14.990000,13.280000,13.060000,0.00,15.68,10.38,0.00,...,28.660000,26.020000,27.530001,24.490000,26.170000,28.010000,21.010000,31.51,24.730000,24.180000
1438,532639.0,23.809999,24.330000,20.030001,12.090000,17.870001,0.00,20.07,0.00,0.00,...,73.550003,78.639999,94.910004,52.840000,79.459999,40.830002,43.630001,31.51,69.180000,52.560001
1439,532640.0,4.200000,5.130000,3.140000,3.360000,3.830000,3.09,0.00,2.20,1.80,...,3.990000,4.020000,4.750000,4.270000,5.220000,4.310000,5.060000,4.71,7.960000,8.350000
1440,532641.0,1.340000,1.790000,0.960000,1.200000,1.160000,0.58,0.00,0.00,0.75,...,0.910000,0.920000,0.900000,0.850000,0.950000,0.850000,0.760000,0.65,1.000000,0.840000


# Map sample data

In [ ]:
wbesD.unary_union.centroid.x

In [ ]:
# Generate h3 grid around wbes data
m = folium.Map(location=[wbesD.unary_union.centroid.y, wbesD.unary_union.centroid.x], zoom_start=7)

wbes_locations = folium.GeoJson(mapping(wbesD.unary_union), style_function=lambda feature: {
    'color':'blue',
    'weight':0.5
})
wbes_locations.add_to(m)

m


In [ ]:
# Generate h3 grid around MWI data
m = folium.Map(location=[mwiD.unary_union.centroid.y, mwiD.unary_union.centroid.x], zoom_start=7)

wbes_locations = folium.GeoJson(mapping(mwiD.unary_union), style_function=lambda feature: {
    'color':'blue',
    'weight':0.5
})
wbes_locations.add_to(m)

m
